# Elhub API data - Gridloss - Summerproject 2025

## Visualization of Price area

Bjørn Eirik Rognskog Nordbak

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "LOSS_PER_MBA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/price-areas"
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    raw = resp.json().get("data", [])
    if not raw:
        print(f"  → no data for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()

    df = pd.json_normalize(
        raw,
        record_path=["attributes", "lossPerMbaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })

    # parse as UTC, then convert to Oslo
    df["startTime"]       = pd.to_datetime(df["startTime"],       utc=True).dt.tz_convert(oslo)
    df["endTime"]         = pd.to_datetime(df["endTime"],         utc=True).dt.tz_convert(oslo)
    df["lastUpdatedTime"] = pd.to_datetime(df["lastUpdatedTime"], utc=True).dt.tz_convert(oslo)

    return df

# loop exactly as before
span_start = datetime(2022, 1, 1, tzinfo=oslo)
span_end   = datetime(2025, 6, 1, tzinfo=oslo)
window     = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    all_chunks.append(fetch_window(cur, nxt))
    cur = nxt

big_df = pd.concat(all_chunks, ignore_index=True)
print("Total rows fetched:", len(big_df))


In [ ]:
big_df

## Data quality

In [ ]:
import pandas as pd

def data_quality_report(df):
    report = pd.DataFrame({
        'column': df.columns.values,
        'dtype': df.dtypes.astype(str).values,
        'non_null_count': df.notnull().sum().values,
        'missing_count': df.isnull().sum().values,
        'missing_percent': (df.isnull().mean() * 100).round(2).values,
        'unique_count': df.nunique(dropna=False).values
    })
    return report

dq_report = data_quality_report(big_df)
dq_report


### Summary for each priceArea - data quality and statistics

In [ ]:
import pandas as pd

for area, grp in big_df.groupby('priceArea'):
    print(f"\n=== Summary for priceArea = {area} ===\n")
    desc = grp.describe()

    # build a dict of formats:
    #   * for the fraction column we want plain float with 2 decimals
    #   * for _every other_ column we want scientific notation with 2 decimals
    fmt = {col: "{:.2e}" for col in desc.columns}
    fmt['calculateLossFraction'] = "{:.2f}"

    # now display with those formats
    display(
        desc.style
            .format(fmt)
            .set_caption("Numeric summary")
    )


### Saving tables to .tex for import to report

In [ ]:
import os
import pandas as pd

# 0) ensure your output folder exists
os.makedirs("tables", exist_ok=True)

# assume big_df is already in your namespace
for area, grp in big_df.groupby("priceArea"):
    # 1) compute the standard describe
    desc = grp.describe()

    # 1a) escape every '%' in the row‐labels to '\%'
    #     so that LaTeX sees a literal percent sign
    desc.rename(
        index=lambda idx: idx.replace("%", r"\%"),
        inplace=True
    )

    # 2) build per‐column formatter functions
    formatters = {
        col: (
            lambda x, col=col: f"{x:.2f}"
            if col == "calculateLossFraction"
            else f"{x:.2e}"
        )
        for col in desc.columns
    }

    # 3) dump *only* the tabular (no \begin{table}… wrapper),
    #    and set escape=False so pandas doesn’t try to re-escape your '\%'
    tex = desc.to_latex(
        formatters=formatters,
        float_format=None,
        na_rep="–",
        column_format="l" + "r" * len(desc.columns),
        bold_rows=True,
        longtable=False,
        index_names=False,  # no blank “index‐name” row
        escape=False        # preserve our manual \% in the row‐labels
    )

    # 4) write out one .tex per area
    out_fname = os.path.join("tables", f"summary_{area}.tex")
    with open(out_fname, "w") as f:
        f.write(tex)
    print(f"Wrote {out_fname}")


## Visualizations

### Scatter plot of production vs. consumption

In [ ]:
import os
import pandas as pd
import plotly.express as px

# ─── MAKE OUTPUT FOLDER ───────────────────────────────────────────────────────
OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# ─── FONT & COLOR SETTINGS ───────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
FONT_FAMILY   = "Roboto"

elhub_colors = [
    "#212148",  # Mørk Lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── PREPARE DATA ─────────────────────────────────────────────────────────────
df = big_df.copy()
df['startTime'] = (
    pd.to_datetime(df['startTime'], utc=True)
      .dt.tz_convert('Europe/Oslo')
)

span_start = df['startTime'].min().date().isoformat()
span_end   = df['startTime'].max().date().isoformat()
title_base = f"Consumption vs Production (kWh) per Price Area\n({span_start} to {span_end})"

MAX_POINTS = 5000
def sample_group(g):
    return g.sample(n=min(len(g), MAX_POINTS), random_state=1)

areas = df['priceArea'].unique().tolist()

# Compute global production range for axis
prod_min = df['productionQuantityKwh'].min()
prod_max = df['productionQuantityKwh'].max()

# ─── PER-AREA SCATTERS ────────────────────────────────────────────────────────
for area in areas:
    dfa = sample_group(df[df['priceArea'] == area])
    idx = areas.index(area)
    color = elhub_colors[idx % len(elhub_colors)]
    
    fig = px.scatter(
        dfa,
        x='productionQuantityKwh',
        y='consumptionQuantityKwh',
        title=f"{title_base} — {area}",
        labels={
            'productionQuantityKwh': 'Production (kWh)',
            'consumptionQuantityKwh': 'Consumption (kWh)'
        },
        color_discrete_sequence=[color]
    )
    
    fig.update_xaxes(range=[prod_min, prod_max])
    fig.update_layout(
        title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
        title_x=0.5,
        font=dict(family=FONT_FAMILY),
        xaxis=dict(
            title='Production (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        yaxis=dict(
            title='Consumption (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        margin=dict(t=100, b=80),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.show()
    
    fname = f"scatter_loss_per_mba_hour_{area.lower()}.pdf"
    path  = os.path.join(OUTDIR, fname)
    fig.write_image(path, format="pdf", width=1000, height=600, scale=1)
    print(f"✅ Saved: {path}")

# ─── COMBINED SCATTER ─────────────────────────────────────────────────────────
df_sample = df.groupby('priceArea', group_keys=False).apply(sample_group)

fig_all = px.scatter(
    df_sample,
    x='productionQuantityKwh',
    y='consumptionQuantityKwh',
    color='priceArea',
    title=f"{title_base} — All Areas",
    labels={
        'productionQuantityKwh': 'Production (kWh)',
        'consumptionQuantityKwh': 'Consumption (kWh)'
    },
    color_discrete_sequence=elhub_colors
)

fig_all.update_xaxes(range=[prod_min, prod_max])
fig_all.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title='Production (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    yaxis=dict(
        title='Consumption (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_all.show()

combined_fname = "scatter_loss_per_mba_hour_all.pdf"
combined_path  = os.path.join(OUTDIR, combined_fname)
fig_all.write_image(combined_path, format="pdf", width=1000, height=600, scale=1)
print(f"✅ Saved: {combined_path}")


### Scatter plot of production vs consumption - with correlation line and R2 score

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# ─── MAKE OUTPUT FOLDER ───────────────────────────────────────────────────────
OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# ─── FONT & COLOR SETTINGS ───────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
FONT_FAMILY   = "Roboto"

elhub_colors = [
    "#212148",  # Mørk Lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── NAMING PREFIX ─────────────────────────────────────────────────────────────
FILENAME_PREFIX = "scatter_loss_per_mba_hour_prod_consum"

# ─── PREPARE DATA ─────────────────────────────────────────────────────────────
df = big_df.copy()
df['startTime'] = (
    pd.to_datetime(df['startTime'], utc=True)
      .dt.tz_convert('Europe/Oslo')
)

span_start = df['startTime'].min().date().isoformat()
span_end   = df['startTime'].max().date().isoformat()
title_base = f"Consumption vs Production (kWh) per Price Area\n({span_start} to {span_end})"

MAX_POINTS = 5000
def sample_group(g):
    return g.sample(n=min(len(g), MAX_POINTS), random_state=1)

areas = df['priceArea'].unique().tolist()

# Global prod range
prod_min, prod_max = df['productionQuantityKwh'].min(), df['productionQuantityKwh'].max()

def make_regression_trace(x, y, color):
    """Fit linear model, return line trace and R²."""
    # reshape for sklearn
    X = x.values.reshape(-1, 1)
    Y = y.values
    model = LinearRegression().fit(X, Y)
    r2 = model.score(X, Y)
    
    # create line points
    xs = np.array([prod_min, prod_max])
    ys = model.predict(xs.reshape(-1,1))
    
    line = go.Scatter(
        x=xs, y=ys,
        mode='lines',
        line=dict(color=color, dash='dash'),
        name=f"Fit (R²={r2:.2f})",
        hoverinfo='none'
    )
    return line, r2

# ─── PER-AREA PLOTS ───────────────────────────────────────────────────────────
for area in areas:
    dfa = sample_group(df[df['priceArea'] == area])
    idx = areas.index(area)
    color = elhub_colors[idx % len(elhub_colors)]
    
    # Build scatter
    scatter = go.Scatter(
        x=dfa['productionQuantityKwh'],
        y=dfa['consumptionQuantityKwh'],
        mode='markers',
        marker=dict(color=color, size=6, opacity=0.6),
        name=area
    )
    
    # Regression
    line, r2 = make_regression_trace(
        dfa['productionQuantityKwh'],
        dfa['consumptionQuantityKwh'],
        color
    )
    
    fig = go.Figure([scatter, line])
    fig.update_layout(
        title=f"{title_base} — {area} (R²={r2:.2f})",
        title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
        title_x=0.5,
        xaxis=dict(
            title='Production (kWh)',
            range=[prod_min, prod_max],
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        yaxis=dict(
            title='Consumption (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        font=dict(family=FONT_FAMILY),
        margin=dict(t=120, b=80),
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False
    )
    
    fig.show()
    
    fname = f"{FILENAME_PREFIX}_{area.lower()}.pdf"
    path  = os.path.join(OUTDIR, fname)
    fig.write_image(path, format="pdf", width=1000, height=600, scale=1)
    print(f"✅ Saved: {path}")

# ─── COMBINED PLOT ────────────────────────────────────────────────────────────
df_samp = df.groupby('priceArea', group_keys=False).apply(sample_group)

# Scatter traces for each area
traces = []
for idx, area in enumerate(areas):
    dfa = df_samp[df_samp['priceArea'] == area]
    col = elhub_colors[idx % len(elhub_colors)]
    traces.append(
        go.Scatter(
            x=dfa['productionQuantityKwh'],
            y=dfa['consumptionQuantityKwh'],
            mode='markers',
            marker=dict(color=col, size=6, opacity=0.6),
            name=area
        )
    )

# Global regression on combined sample
comb_line, comb_r2 = make_regression_trace(
    df_samp['productionQuantityKwh'],
    df_samp['consumptionQuantityKwh'],
    '#000000'  # black for combined fit line
)
traces.append(comb_line)

fig_all = go.Figure(traces)
fig_all.update_layout(
    title=f"{title_base} — All Areas (R²={comb_r2:.2f})",
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    xaxis=dict(
        title='Production (kWh)',
        range=[prod_min, prod_max],
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    yaxis=dict(
        title='Consumption (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    font=dict(family=FONT_FAMILY),
    margin=dict(t=120, b=80),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_all.show()

combined_fname = f"{FILENAME_PREFIX}_all.pdf"
combined_path  = os.path.join(OUTDIR, combined_fname)
fig_all.write_image(combined_path, format="pdf", width=1000, height=600, scale=1)
print(f"✅ Saved: {combined_path}")


### Scatter plot of loss fraction vs. price area

In [ ]:
import matplotlib.pyplot as plt

areas = big_df['priceArea'].astype('category')
codes = areas.cat.codes

plt.figure(figsize=(10, 6))
plt.scatter(codes, big_df['calculateLossFraction'], alpha=0.6)

# add a horizontal dotted line at zero
plt.axhline(0, linestyle='--', linewidth=1, color='gray')

plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Loss Fraction by Price Area')

plt.xticks(
    ticks=range(len(areas.cat.categories)),
    labels=areas.cat.categories,
    rotation=45,
    ha='right'
)

plt.tight_layout()
plt.show()


### boxplot of loss fraction by price area

In [ ]:
import matplotlib.pyplot as plt

# 1) group your loss fractions by area
groups = [
    grp['calculateLossFraction'].values
    for _, grp in big_df.groupby('priceArea')
]
labels = [
    name
    for name, _ in big_df.groupby('priceArea')
]

# 2) draw the boxplot
plt.figure(figsize=(12, 6))
plt.boxplot(groups, labels=labels, notch=True, patch_artist=True)

# 3) add a horizontal zero-loss line
plt.axhline(0, linestyle='--', linewidth=1, color='gray')

plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Distribution of Loss Fraction by Price Area')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### Violin plot of loss fractions by price area

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.violinplot(
    x='priceArea',
    y='calculateLossFraction',
    hue='priceArea',         # now palette is valid
    data=big_df,
    inner='quartile',
    cut=0,
    density_norm='width',    # new name for scale='width'
    palette='pastel',
    dodge=False,             # don’t offset the “hue” violins
    legend=False             # don’t draw the extra legend
)

plt.axhline(0, linestyle='--', linewidth=1, color='gray')
plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Violin Plot of Loss Fraction by Price Area')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### Violin plot of loss fractions by price area - cleaned outliers

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def remove_outliers_iqr(df, value_col, group_col):
    """
    Remove rows where value_col is outside [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
    calculated within each group of group_col.
    """
    def _filter(group):
        q1 = group[value_col].quantile(0.25)
        q3 = group[value_col].quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        return group[(group[value_col] >= lower) & (group[value_col] <= upper)]
    
    # Apply per-group filtering and re-concatenate
    return df.groupby(group_col, group_keys=False).apply(_filter)

# 1) Filter out outliers
clean_df = remove_outliers_iqr(
    big_df,
    value_col='calculateLossFraction',
    group_col='priceArea'
)

# 2) Plot the violin plot on the cleaned data
plt.figure(figsize=(12, 6))
sns.violinplot(
    x='priceArea',
    y='calculateLossFraction',
    data=clean_df,
    hue='priceArea',         # allows using a pastel palette
    inner='quartile',         # show quartile lines
    cut=0,                    # no density beyond data range
    density_norm='width',     # uniform max width
    palette='pastel',
    dodge=False,
    legend=False
)

# 3) Zero-loss reference line
plt.axhline(0, linestyle='--', linewidth=1, color='gray')

# 4) Labels & layout
plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Violin Plot of Loss Fraction by Price Area\n(after removing outliers)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
